In [1]:
import pandas as pd
import os
import glob
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
train_files = glob.glob('data/data_stage_one/training_set/*.txt')
test_files = glob.glob('data/data_stage_one/test_set/*.txt')

In [3]:

files = {'train': train_files,'test': test_files}

In [5]:
data = []

for dtype, file in files.items():
    for i in range(len(file)):
        f = open(file[i], 'r')
        f_text = f.read()
        #print(f_text)
        f.close()
        party = file[i].split('_')[-1][0]
        sample_group = dtype.split('_')[-1]
        #print(review_sentiment)
        review_label = 0 if party == 'D' else 1
        #print(review_label)
        data.append([f_text, party, sample_group, review_label])


In [6]:
data = pd.DataFrame(data)

In [7]:
convote_data = data.rename(index=str, columns={0: 'text', 1: 'party', 2: 'group', 3: 'party_label'})

In [8]:
convote_data.tail()

,text,party,group,party_label
7414,"mr. chairman , i demand a recorded vote . \n",R,test,1
7415,"mr. speaker , i yield 1 minute to the gentlema...",D,test,0
7416,"mr. chairman , i yield 5 minutes to the gentle...",R,test,1
7417,"mr. chairman , i yield 1 1/2 minutes to the ge...",R,test,1
7418,"mr. speaker , i thank the gentleman from north...",D,test,0


In [9]:

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):
 
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split()) 
    return text
    

convote_data['text'] = convote_data['text'].apply(clean_text)

In [10]:
X = convote_data.text
y = convote_data.party_label
tf_idf = TfidfVectorizer(stop_words = 'english')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
X_train_fit = tf_idf.fit_transform(X_train)
X_test_fit = tf_idf.transform(X_test)

In [11]:
model = LogisticRegression()
model.fit(X_train_fit,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
y_predict = model.predict(X_test_fit)

In [13]:
Y_predict_train = model.predict(X_train_fit)
score_train = accuracy_score(y_train, Y_predict_train)
score_test = accuracy_score(y_test, y_predict)

In [15]:
score_train

0.8544229149115417

In [14]:
score_test

0.7574123989218329